In [1]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass


from pydantic_ai import Agent
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.azure import AzureProvider

model = OpenAIChatModel("gpt-4.1-mini", provider=AzureProvider())

agent = Agent(
    model=model,
    instructions="You are an expert when it comes to writing complete and detailed docstrings from not very informative tool definitions." \
    "Your task is to improve the tool descriptions by infering their intended use from the context and from the parameter json schema." \
    "All these tools are used for the Ally Portal, which is a web application to manage Copilot configurations." \
    "Make sure the docstrings follow google style formatting guidelines." \
    "This is important: Only answer with the new docstring, do not include any other text!", 
)




In [2]:
from lib.openapi_to_tools import OpenAPIToolsLoader

# Initialize the loader
loader = OpenAPIToolsLoader(
    openapi_url="https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json"
)

# Load all tools
print("Loading tools from OpenAPI spec...")
tools = loader.load_tools()

Loading tools from OpenAPI spec...
Generating models file 'api_models.py' from https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json...
Generating models file 'api_models.py' from https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json...
Successfully generated 'api_models.py'
Created tool: get_aws_logs_api_getAWSLogs_post [POST /api/getAWSLogs]
Created tool: get_capabilities_api_capabilities_get [GET /api/capabilities]
Created tool: get_active_config_api_getActiveConfig_post [POST /api/getActiveConfig]
Created tool: validate_config_api_validateConfig_post [POST /api/validateConfig]
Created tool: set_config_api_setConfig_post [POST /api/setConfig]
Created tool: get_config_history_api_getConfigHistory_post [POST /api/getConfigHistory]
Created tool: create_endpoint_api_createEndpoint_post [POST /api/createEndpoint]
Created tool: get_endpoint_metadata_api_getEndpointMetadata_post [POST /api/getEndpointMetadata]
Created tool: update_endpoint_metadata_api_updateEnd

In [3]:
for tool in tools:
    print(f"{tool.tool_def}\n")

ToolDefinition(name='get_aws_logs_api_getAWSLogs_post', parameters_json_schema={'type': 'object', 'properties': {'log_group': {'type': 'string', 'title': 'Log Group', 'description': 'Log group in AWS'}, 'session_id': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Session Id', 'description': 'Session ID for filtering logs'}, 'start_time': {'type': 'string', 'title': 'Start Time', 'description': 'Start time for log retrieval'}, 'end_time': {'type': 'string', 'title': 'End Time', 'description': 'End time for log retrieval'}}, 'required': ['log_group', 'start_time', 'end_time'], 'additionalProperties': False}, description='Retrieve AWS log data.\n\nThis function retrieves log data from a specified AWS log group, filtering\nthe logs based on session ID and time range.  It returns a list of log\nentries along with a dictionary of session IDs and their corresponding\ntimestamps.\n\nReturns:\n    A response containing the log data and session IDs.\nRaises:\n    NoSuccess: If the l

In [4]:
special_tool=loader.get_tool_by_operation_id("validate_config_api_validateConfig_post")
special_tool.tool_def.parameters_json_schema

{'type': 'object',
 'properties': {'endpoint': {'type': 'string',
   'title': 'Endpoint',
   'description': 'Name of the endpoint'},
  'config_data': {'properties': {'dep_name': {'type': 'string',
     'title': 'Dep Name',
     'description': "OpenAI model deployment name, such as 'gpt-4o-mini'"},
    'temperature': {'type': 'number',
     'title': 'Temperature',
     'description': 'Temperature for the LLM, default is 0. Set to a negative value to use the default temperature of the model.',
     'default': 0.0},
    'history_reduction': {'anyOf': [{'properties': {'old_calls': {'anyOf': [{'type': 'boolean'},
          {'type': 'null'}],
         'title': 'Old Calls'},
        'bad_calls': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
         'title': 'Bad Calls'},
        'large_api_response': {'anyOf': [{'type': 'boolean'},
          {'type': 'null'}],
         'title': 'Large Api Response'}},
       'additionalProperties': False,
       'type': 'object',
       'title': 'Histor

In [5]:
# Collect improved docstrings and write to markdown file
improved_docstrings = []

for tool in tools:
    tool_def = tool.tool_def
    new_docstring = await agent.run(f"Generate a new improved docstring for me:\n\n{tool_def}")
    result = new_docstring.output.strip()
    
    improved_docstrings.append({
        'name': tool.name,
        'docstring': result
    })
    
    print(f"Processed tool: {tool.name}")

print(f"\nProcessed {len(improved_docstrings)} tools total.")

Processed tool: get_aws_logs_api_getAWSLogs_post
Processed tool: get_capabilities_api_capabilities_get
Processed tool: get_capabilities_api_capabilities_get
Processed tool: get_active_config_api_getActiveConfig_post
Processed tool: get_active_config_api_getActiveConfig_post
Processed tool: validate_config_api_validateConfig_post
Processed tool: validate_config_api_validateConfig_post
Processed tool: set_config_api_setConfig_post
Processed tool: set_config_api_setConfig_post
Processed tool: get_config_history_api_getConfigHistory_post
Processed tool: get_config_history_api_getConfigHistory_post
Processed tool: create_endpoint_api_createEndpoint_post
Processed tool: create_endpoint_api_createEndpoint_post
Processed tool: get_endpoint_metadata_api_getEndpointMetadata_post
Processed tool: get_endpoint_metadata_api_getEndpointMetadata_post
Processed tool: update_endpoint_metadata_api_updateEndpointMetadata_post
Processed tool: update_endpoint_metadata_api_updateEndpointMetadata_post
Process

In [6]:
# Write improved docstrings to markdown file
markdown_content = "# Improved Tool Docstrings\n\n"
markdown_content += "This document contains improved docstrings for tools generated from the Ally Portal OpenAPI specification.\n\n"
markdown_content += "---\n\n"

for item in improved_docstrings:
    markdown_content += f"## {item['name']}\n\n"
    markdown_content += f"```\n{item['docstring']}\n```\n\n"
    markdown_content += "---\n\n"

# Write to file
with open("ally_config_improved_tool_descriptions.md", "w") as f:
    f.write(markdown_content)

print("✅ Markdown file 'ally_config_improved_tool_descriptions.md' has been created successfully!")
print(f"📄 Contains improved docstrings for {len(improved_docstrings)} tools.")

✅ Markdown file 'ally_config_improved_tool_descriptions.md' has been created successfully!
📄 Contains improved docstrings for 36 tools.
